# MTA Traffic Forecasting
#### Justin Morgan & Khyatee Desai
This notebook includes the data collection, storage, and cleaning process. The data is sourced from the [MTA turnstile data archive](http://web.mta.info/developers/turnstile.html), which all turnstile data from all NYC subway stations in comma delineated text files, segmented by week.
# Part 1: Web Scrape MTA Data
The data is iteratively scraped from the webpage using BeautifulSoup, and the text files are currently stored locally within a "data" folder.
<br><br>
Future steps will involve storing this data within an S3 bucket in the AWS cloud for faster storage & retrieval.

In [115]:
# import necessary packages
import pandas as pd
import numpy as np
import os
import requests
import urllib.request
import time
from timeit import default_timer as timer
import humanfriendly
from bs4 import BeautifulSoup
from datetime import datetime as dt
import pickle

### Get MTA turnstile data from publicly available website

In [102]:
url_root = r'http://web.mta.info/developers/' # set root url
# path = r'/Users/justinwilliams/projects/mta_turnstile/data/' # commented this out and changed it below to just save to "./data/" folder so it works for both of us - k
starttime = timer() # start timer to time process

req = requests.get(url_root + 'turnstile.html') # send request
soup = BeautifulSoup(req.content, 'html.parser') # parse html and save to bs4 object
weekly_data = soup.find(class_='span-84 last') # find class_ where file links are locatedb

## This way took much longer so used urllib.request.urlretrieve
# for file in weekly_data.findAll('a'):
#     print('Saving file turnstile ' + str(file)[39:49])
#     datafile = requests.get(url_root + str(file)[9:49])
#     with open(path + str(file)[39:49], 'w') as outf:
#         for line in datafile.text:
#             outf.writelines(line)
#     time.sleep(1)

# endtime = timer()
# print('Completed in ' + humanfriendly.format_timespan(endtime-starttime))

counter = 1
while counter <= 5:
    for one_a_tag in weekly_data.findAll('a')[:5]: #just using first 5 files for now
        file = one_a_tag['href']
        datafile = url_root + file
        urllib.request.urlretrieve(datafile, './data/' + datafile[datafile.find('/turnstile_')+1:])
        time.sleep(1)
        counter += 1
        print('Saving file turnstile ' + str(one_a_tag)[39:49])
    
endtime = timer()
print('Completed in ' + humanfriendly.format_timespan(endtime-starttime))

Saving file turnstile 210313.txt
Saving file turnstile 210306.txt
Saving file turnstile 210227.txt
Saving file turnstile 210220.txt
Saving file turnstile 210213.txt
Completed in 15.94 seconds


Took __47 minutes and 17.79 seconds__ to download all files, also folder size is __12.05GB__

Maybe there is a better way to do this?

## Web Scrape Station Coordinates from Wikipedia
Station coordinates will be utilzed to visiualize data geospatially.
- <a href="https://en.wikipedia.org/wiki/List_of_New_York_City_Subway_stations_in_Manhattan" target=blank> Manhattan</a>
- <a href="https://en.wikipedia.org/wiki/List_of_New_York_City_Subway_stations_in_Brooklyn" target=blank> Brooklyn</a>
- <a href="https://en.wikipedia.org/wiki/List_of_New_York_City_Subway_stations_in_Queens" target=blank> Queens</a>
- <a href="https://en.wikipedia.org/wiki/List_of_New_York_City_Subway_stations_in_the_Bronx" target=blank> Bronx</a>
- <a href="https://en.wikipedia.org/wiki/Staten_Island_Railway" target=blank> Staten Island</a>

Coordinates are on the upper right hand side of each stations specific page...these can be navigated too through web scraping...prob will have to write a slightly different code module for each borough. lol

# Part 2: Data Cleaning
The below cells entail cleaning, reformatting, and aggregating the data

### Field Descriptions <br>
http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt

`C/A      = Control Area (A002)`<br>
`UNIT     = Remote Unit for a station (R051)`<br>
`SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)`<br>
`STATION  = Represents the station name the device is located at`<br>
`LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.`<br>
`DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   `<br>
`DATE     = Represents the date (MM-DD-YY)`<br>
`TIME     = Represents the time (hh:mm:ss) for a scheduled audit event`<br>
`DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)`<br>
        `1. Audits may occur more that 4 hours due to planning, or troubleshooting activities.`<br>
        `2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered.`<br>
`ENTRIES  = The comulative entry register value for a device`<br>
`EXIST    = The cumulative exit register value for a device`<br>
### Inspect Data

In [104]:
df = pd.read_csv("./data/turnstile_210313.txt")
df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/06/2021,03:00:00,REGULAR,7540642,2572027
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/06/2021,07:00:00,REGULAR,7540645,2572030
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/06/2021,11:00:00,REGULAR,7540676,2572093
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/06/2021,15:00:00,REGULAR,7540764,2572128
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/06/2021,19:00:00,REGULAR,7540904,2572160
...,...,...,...,...,...,...,...,...,...,...,...
209312,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,03/12/2021,04:00:00,REGULAR,5554,544
209313,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,03/12/2021,08:00:00,REGULAR,5554,544
209314,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,03/12/2021,12:00:00,REGULAR,5554,544
209315,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,03/12/2021,16:00:00,REGULAR,5554,544


## Concatenate Files
Concatenate each data file into a Pandas dataframe

In [105]:
# create main df using first file in the folder
for filename in os.listdir("./data/")[:1]:
    df = pd.read_csv("./data/"+filename)

# iterate over remaining files in the folder and concat them to main df
for filename in os.listdir("./data/")[1:]:
    df = pd.concat([df,pd.read_csv("./data/"+filename)])

In [106]:
df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/06/2021,03:00:00,REGULAR,7527244,2565995
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/06/2021,07:00:00,REGULAR,7527246,2566004
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/06/2021,11:00:00,REGULAR,7527296,2566054
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/06/2021,15:00:00,REGULAR,7527430,2566098
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/06/2021,19:00:00,REGULAR,7527588,2566129
...,...,...,...,...,...,...,...,...,...,...,...
209039,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/19/2021,04:00:00,REGULAR,5554,544
209040,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/19/2021,08:00:00,REGULAR,5554,544
209041,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/19/2021,12:00:00,REGULAR,5554,544
209042,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/19/2021,16:00:00,REGULAR,5554,544


## Reformat Data Types
Convert strings to DateTime format

In [107]:
df.DATE = pd.to_datetime(df['DATE'])

### Change Time Column to Timestamp Object
*Commented out for now because probably not going to use timestamps, just focusing on date*

In [13]:
# df.TIME = pd.to_datetime(df['TIME'])
# df.TIME.apply(lambda x: dt.timestamp(x))

In [14]:
df.dtypes

C/A                                                                             object
UNIT                                                                            object
SCP                                                                             object
STATION                                                                         object
LINENAME                                                                        object
DIVISION                                                                        object
DATE                                                                    datetime64[ns]
TIME                                                                            object
DESC                                                                            object
ENTRIES                                                                          int64
EXITS                                                                            int64
dtype: object

### Inspect a specific station & date

In [108]:
df[(df.STATION == 'HALSEY ST') & (df.DATE.astype(str)=='2021-02-15')].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
36293,H028,R266,00-00-00,HALSEY ST,L,BMT,2021-02-15,03:00:00,REGULAR,5897529,3008491
36294,H028,R266,00-00-00,HALSEY ST,L,BMT,2021-02-15,07:00:00,REGULAR,5897606,3008499
36295,H028,R266,00-00-00,HALSEY ST,L,BMT,2021-02-15,11:00:00,REGULAR,5897771,3008564
36296,H028,R266,00-00-00,HALSEY ST,L,BMT,2021-02-15,15:00:00,REGULAR,5897864,3008636
36297,H028,R266,00-00-00,HALSEY ST,L,BMT,2021-02-15,19:00:00,REGULAR,5897921,3008755


## Derive Daily Entries
Entries and Exits are currently listed cumulatively. The below process groups the dataframe by station, date, SCP, UNIT, and C/A on Max and Min, then subtracts the two to find the number of turnstile entries per day at each station.

*....not really sure what SCP, UNIT, and C/A are but i think they might indicate an individual tracker device at a station for each turnstile? idkidk*

In [110]:
# min gives us cumulative entries at the beginning of each day
df.groupby(['STATION','DATE','SCP','UNIT','C/A'])[['ENTRIES']].min()

ENTRIES
STATION   DATE       SCP      UNIT C/A            
1 AV      2021-02-06 00-00-00 R248 H007   15524923
                     00-00-01 R248 H007   61232032
                     00-03-00 R248 H007  370878741
                     00-03-01 R248 H007    2615699
                     00-03-02 R248 H007    6659920
...                                            ...
ZEREGA AV 2021-03-12 00-00-01 R326 R419     227376
                     00-03-00 R326 R419    1142551
                     00-03-01 R326 R419    1218309
                     00-05-00 R326 R419        232
                     00-05-01 R326 R419         39

[174735 rows x 1 columns]

In [111]:
# max gives us cumulative entries at the end of each day
df.groupby(['STATION','DATE','SCP','UNIT','C/A'])[['ENTRIES']].max()

ENTRIES
STATION   DATE       SCP      UNIT C/A            
1 AV      2021-02-06 00-00-00 R248 H007   15525120
                     00-00-01 R248 H007   61232338
                     00-03-00 R248 H007  370878782
                     00-03-01 R248 H007    2615731
                     00-03-02 R248 H007    6660014
...                                            ...
ZEREGA AV 2021-03-12 00-00-01 R326 R419     227473
                     00-03-00 R326 R419    1142796
                     00-03-01 R326 R419    1218850
                     00-05-00 R326 R419        232
                     00-05-01 R326 R419         39

[174735 rows x 1 columns]

In [112]:
# subtract min from max to get number of entries each day
grouped = df.groupby(['STATION','DATE','SCP','UNIT','C/A'])[['ENTRIES']].max()- df.groupby(['STATION','DATE','SCP','UNIT','C/A'])[['ENTRIES']].min()
grouped


ENTRIES
STATION   DATE       SCP      UNIT C/A          
1 AV      2021-02-06 00-00-00 R248 H007      197
                     00-00-01 R248 H007      306
                     00-03-00 R248 H007       41
                     00-03-01 R248 H007       32
                     00-03-02 R248 H007       94
...                                          ...
ZEREGA AV 2021-03-12 00-00-01 R326 R419       97
                     00-03-00 R326 R419      245
                     00-03-01 R326 R419      541
                     00-05-00 R326 R419        0
                     00-05-01 R326 R419        0

[174735 rows x 1 columns]

### Sum up Devices

In [113]:
# sum up all of the devices at a station to get total daily entries
grouped_entries = grouped.groupby(['STATION', 'DATE']).sum()
grouped_entries

ENTRIES
STATION   DATE               
1 AV      2021-02-06     4295
          2021-02-07     2559
          2021-02-08     5636
          2021-02-09     5638
          2021-02-10     5995
...                       ...
ZEREGA AV 2021-03-08      918
          2021-03-09      979
          2021-03-10      976
          2021-03-11     1051
          2021-03-12     1015

[13225 rows x 1 columns]

### Pivot Table

In [119]:
# pivot the dataframe so date on the columns
entries_df = grouped_entries.pivot_table(index='DATE', columns='STATION')
entries_df.columns = entries_df.columns.droplevel(0) # drop unnecessary multindex level
entries_df

STATION,1 AV,103 ST,103 ST-CORONA,104 ST,110 ST,111 ST,116 ST,116 ST-COLUMBIA,121 ST,125 ST,...,WHITEHALL S-FRY,WHITLOCK AV,WILSON AV,WINTHROP ST,WOODHAVEN BLVD,WOODLAWN,WORLD TRADE CTR,WTC-CORTLANDT,YORK ST,ZEREGA AV
DATE,,,,,,,,,,,,,,,,,,,,,
2021-02-06,4295.0,5498.0,7324.0,1076.0,2551.0,5350.0,7027.0,1322.0,672.0,16654.0,...,1725.0,515.0,1067.0,1383.0,6103.0,1974.0,132.0,979.0,1.0,539.0
2021-02-07,2559.0,3432.0,4676.0,663.0,1551.0,3245.0,4162.0,636.0,424.0,9176.0,...,995.0,317.0,678.0,775.0,3349.0,929.0,73.0,469.0,0.0,326.0
2021-02-08,5636.0,8697.0,9175.0,1844.0,3620.0,7221.0,9372.0,2068.0,904.0,21757.0,...,2887.0,743.0,1478.0,2848.0,8812.0,2660.0,2274.0,1524.0,2034.0,928.0
2021-02-09,5638.0,8926.0,9150.0,1890.0,3797.0,7355.0,9814.0,2115.0,924.0,21995.0,...,2943.0,786.0,1534.0,2920.0,8986.0,2825.0,2318.0,1511.0,2124.0,940.0
2021-02-10,5995.0,9227.0,9461.0,1834.0,3945.0,7599.0,10247.0,2243.0,1034.0,23161.0,...,3026.0,823.0,1621.0,3005.0,9350.0,2783.0,2387.0,1587.0,2290.0,922.0
2021-02-11,5927.0,9013.0,9798.0,1936.0,3803.0,7215.0,9592.0,2210.0,947.0,22808.0,...,2780.0,794.0,1558.0,2914.0,8894.0,2934.0,2409.0,1560.0,2245.0,927.0
2021-02-12,6044.0,8845.0,9717.0,1816.0,3594.0,7596.0,9487.0,2242.0,998.0,22780.0,...,2755.0,738.0,1610.0,2764.0,8911.0,2918.0,2324.0,1544.0,2198.0,892.0
2021-02-13,4112.0,5195.0,7207.0,1123.0,2485.0,5127.0,6709.0,1128.0,616.0,15367.0,...,1488.0,428.0,1106.0,1338.0,5775.0,1867.0,138.0,1204.0,0.0,506.0
2021-02-14,3343.0,4030.0,5439.0,765.0,1722.0,3940.0,5412.0,853.0,482.0,11613.0,...,1209.0,345.0,934.0,961.0,4238.0,1290.0,102.0,837.0,0.0,399.0


### Pickle Cleaned Data

In [120]:
# entries_df.to_pickle("./pickled/cleaned_data")